In [1]:
import os
import hw
import numpy as np
import env_ingredient_add as env
from hw import PoseTrajectorySource
import trajectories as traj
from importlib import reload

def get_scene():
    if os.getcwd() == "/datasets/_deepnote_work/manipulation/project": 
        #scene = open("/work/manipulation/project/objects/environment_setup.yaml")
        scene = env.get_environment_set_up(no_scene=False,include_driver=True)
        xmls = [os.getcwd() + "/package.xml", "/work/manipulation/project/package.xml"]
    else:
        #scene = open("objects/environment_setup.yaml") # local setup
        scene = env.get_environment_set_up(no_scene=False,include_driver=True)
        xmls = [os.getcwd() + "/package.xml"]

    return scene

In [2]:
scenario = hw.LoadScenario(data=get_scene())
meshcat = hw.StartMeshcat()

INFO:drake:Meshcat listening for connections at https://92fbdad2-a4a9-4ef8-b493-c307060b34a1.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


### trajectory 0: iiwa_painter

In [3]:
print(scenario)

Scenario(random_seed=0, simulation_duration=inf, simulator_config=SimulatorConfig(integration_scheme='runge_kutta3', max_step_size=0.01, accuracy=0.01, use_error_control=False, target_realtime_rate=0.0, publish_every_time_step=False), plant_config=MultibodyPlantConfig(time_step=0.01, penetration_allowance=0.001, stiction_tolerance=0.001, contact_model='hydroelastic_with_fallback', discrete_contact_approximation='sap', discrete_contact_solver='', sap_near_rigid_threshold=1.0, contact_surface_representation='polygon', adjacent_bodies_collision_filters=True), directives=[ModelDirective(add_model=AddModel(file='package://manipulation/mobile_iiwa14_primitive_collision.urdf', name='mobile_iiwa', default_joint_positions={'iiwa_base_x': array([-1.]), 'iiwa_base_y': array([1.]), 'iiwa_base_z': array([0.]), 'iiwa_joint_1': array([-1.57]), 'iiwa_joint_2': array([0.1]), 'iiwa_joint_3': array([0.]), 'iiwa_joint_4': array([-1.2]), 'iiwa_joint_5': array([0.]), 'iiwa_joint_6': array([1.6]), 'iiwa_join

block below takes ~4m to run -> can we bring it down?

In [4]:
# reload(env)
# reload(hw)

# at present, trajectory blank doesn't work
diagram,sim = hw.init_diagram(meshcat,scenario)
# hw.print_diagram(diagram)

context = diagram.CreateDefaultContext()
traj0 = hw.create_painter_trajectory(diagram,meshcat,context)

meshcat.Delete()
diagram,sim = hw.init_diagram(meshcat,scenario,traj0)
controller = diagram.GetSubsystemByName("diff_ik_integrator")


INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/69c92595a391eb023c27ab6ac8f80d58a3e4612d.tar.gz


RuntimeError: The OBJ file contains multiple objects; only OBJs with a single object are supported: File name: '/datasets/_deepnote_work/manipulation/project/objects/ovenV1.obj'

sim runs in 1min -> oven not fixed yet, with 1 ingredient each 

In [5]:
hw.fix_base_pos(controller.get_parameters(), 3*[True]) # z is locked to 25 cm.
# hw.create_painter_trajectory(diagram,meshcat)

hw.run_simulation(meshcat,sim,20)

### trajectory 1: simple move to bowl

In [6]:
# # TODO(mattg): move this into some sort of planner system that can move between states.

# reload(traj)

# diagram,sim = hw.init_diagram(meshcat,scenario)
# traj1 = traj.traj_linear_move_to_bowl_0(diagram)
# meshcat.Delete()

# diagram,sim = hw.init_diagram(meshcat,scenario,traj1)

# # problem: we're missing a way to actually pass new trajectories into the robot.

# # pose_source = diagram.GetSubsystemByName("hw.PoseTrajectorySource@00007fc3283167c0")

In [7]:
# hw.run_simulation(meshcat,sim,20)

## State Machine (WIP)

In [8]:
# class PizzaRobotState(Enum):
#     """FSM state enumeration"""

#     PLAN_IIWA_PAINTER = 0
#     EXECUTE_IIWA_PAINTER = 1
#     FINISHED = 2


# class PizzaPlanner(LeafSystem):
#     def __init__(
#             self,
#             num_joint_positions: int
#         ):
#         """
#         Args: 
#             (blank for now)

#         Adapted from OpenLoopPlanarPushingPlanner (github user nepfaff/iiwa_setup).
#         """
#         super().__init__()

#         self._is_finished = False # encodes that we've reached goal state

#         # internal state
#         self._fsm_state_idx = int(
#             self.DeclareAbstractState(
#                 AbstractValue.Make(PizzaRobotState.PLAN_IIWA_PAINTER)
#             )
#         )
#         """current FSM state"""

#         self._current_iiwa_positions_idx = int(
#             self.DeclareDiscreteState(num_joint_positions)
#         )
#         """The current iiwa positions. These are used to command the robot to stay idle."""

#         self._current_pose_trajectory_idx = int(
#             self.DeclareAbstractState(
#                 AbstractValue.Make(PiecewisePoseWithTimingInformation())
#             )
#         )
#         """The current pose trajectory."""



#         # input ports
#         self._iiwa_state_estimated_input_port = self.DeclareVectorInputPort(
#             "mobile_iiwa.estimated_state", num_joint_positions*2 # positions and velocities
#         )

#         # output ports

#     def _get_current_iiwa_positions(
#         self, context: Context, output: BasicVector
#     ) -> None:
#         positions = context.get_discrete_state(
#             self._current_iiwa_positions_idx
#         ).get_value()
#         output.set_value(positions)

#     def _initialize_discrete_state(
#         self, context: Context, discrete_values: DiscreteValues
#     ) -> None:
#         # Initialize the current iiwa positions
#         discrete_values.set_value(
#             self._current_iiwa_positions_idx,
#             self._iiwa_position_measured_input_port.Eval(context),
#         )

#     # plan move to home location
#     def _plan_iiwa_painter(self,context: Context) -> PathParameterizedTrajectory:
#         pass

#     # iiwa painter trajectory
#     def _execute_iiwa_painter(self, context: Context):
#         pass



#     def _run_fsm_logic(self, context: Context, state: State) -> None:
#         """FSM state transition logic."""
#         current_time = context.get_time()
#         timing_information: OpenLoopPlanarPushingPlanarTimingInformation = (
#             context.get_mutable_abstract_state(self._timing_information_idx).get_value()
#         )

#         mutable_fsm_state = state.get_mutable_abstract_state(self._fsm_state_idx)
#         fsm_state_value: PizzaRobotState = context.get_abstract_state(
#             self._fsm_state_idx
#         ).get_value()

#         if fsm_state_value == PizzaRobotState.PLAN_IIWA_PAINTER:
#             logging.info("Current state: PLAN_IIWA_PAINTER")

#             # for now, this state just inits the robot position at the start time.
            
            


#             # q_traj = self._plan_move_to_start(context)
#             # state.get_mutable_abstract_state(self._current_joint_traj_idx).set_value(
#             #     TrajectoryWithTimingInformation(
#             #         trajectory=q_traj,
#             #         start_time_s=timing_information.start_move_to_start,
#             #     )
#             # )
#             # timing_information.end_move_to_start = (
#             #     timing_information.start_move_to_start + q_traj.end_time()
#             # )
#             # state.get_mutable_abstract_state(self._timing_information_idx).set_value(
#             #     timing_information
#             # )

#             logging.info("Transitioning to EXECUTE_IIWA_PAINTER FSM state.")
#             mutable_fsm_state.set_value(PizzaRobotState.EXECUTE_IIWA_PAINTER)

#         elif fsm_state_value == PizzaRobotState.EXECUTE_IIWA_PAINTER:

#             # this is where we would actually execute the 
#             # traj0 = hw.create_painter_trajectory(diagram,meshcat,context)

#             logging.info("Transitioning to FINISHED FSM state")


#         elif fsm_state_value == PizzaRobotState.FINISHED:

#             self._is_finished = True

#         else:
#             logging.error(f"Invalid FSM state: {fsm_state_value}")
#             exit(1)

    

#     def is_finished(self) -> bool:
#         """Returns True if the task has been completed and False otherwise."""
#         return self._is_finished






## scratch code

### gets the name of the robot's joints

In [9]:

# plant = diagram.GetSubsystemByName("station").GetSubsystemByName("plant")

# for index in plant.GetJointIndices():
#     print(plant.get_joint(index).name())

In [10]:


# print(controller.get_parameters().get_joint_position_limits())

# print(f"after running: {controller.get_parameters().get_joint_position_limits()}")

# hw.run_simulation(meshcat,sim,10)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=92fbdad2-a4a9-4ef8-b493-c307060b34a1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>